In [14]:
# 1.计算网格中心点

import pandas as pd
import numpy as np

def calculate_grid_centers(input_file_path, output_file_path):
    """
    计算网格中心点坐标的函数
    参数:
        input_file_path: 输入CSV文件路径
        output_file_path: 输出CSV文件路径
    """
    # 读取输入文件
    df = pd.read_csv(input_file_path)
    
    # 提取经纬度范围
    longitude_ranges = df['Longitude Range'].apply(lambda x: tuple(map(float, x[1:-1].split(', '))))
    latitude_ranges = df['Latitude Range'].apply(lambda x: tuple(map(float, x[1:-1].split(', '))))
    
    # 计算中心点坐标
    df['Center Longitude'] = (longitude_ranges.apply(lambda x: x[0]) + longitude_ranges.apply(lambda x: x[1])) / 2
    df['Center Latitude'] = (latitude_ranges.apply(lambda x: x[0]) + latitude_ranges.apply(lambda x: x[1])) / 2
    
    # 保存中心点坐标
    df[['Center Latitude', 'Center Longitude']].to_csv(output_file_path, index=False)


In [1]:
# calculate_grid_centers(r'save/NYC_grid_c.csv', 'save/grid_centers_c.csv')
# calculate_grid_centers(r'save/NYC_grid_f.csv', 'save/grid_centers_f.csv')
# calculate_grid_centers(r'save/NYC_grid_uf.csv', 'save/grid_centers_uf.csv')

In [2]:
# 计算flow值（即出租车行程数据，乘客的人流量）
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
import itertools

def calculate_grid_flows(grid_file_path, taxi_zones_file_path, taxi_trips_file_path, start_date, end_date, output_file_path):
    """
    计算网格的流入量和流出量
    参数:
        grid_file_path: 细粒度网格CSV文件路径
        taxi_zones_file_path: 出租车区GeoJSON文件路径
        taxi_trips_file_path: 出租车出行CSV文件路径
        start_date: 起始日期
        end_date: 结束日期
        output_file_path: 输出CSV文件路径
    """
    # 读取细粒度网格数据
    grid_data = pd.read_csv(grid_file_path)
    print("read1")

    # 读取出租车区数据
    taxi_zones = gpd.read_file(taxi_zones_file_path)
    print("read1")

    # 读取出租车出行数据
    taxi_trips = pd.read_csv(taxi_trips_file_path)
    print("read1")

    # 将出租车出行数据按时间段进行分组
    taxi_trips['pickup_datetime'] = pd.to_datetime(taxi_trips['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
    taxi_trips['dropoff_datetime'] = pd.to_datetime(taxi_trips['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')

    # 过滤出指定日期范围内的出租车出行数据
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    taxi_trips = taxi_trips[(taxi_trips['pickup_datetime'] >= start_date) & (taxi_trips['pickup_datetime'] <= end_date)]

    # 过滤出起点和终点不在同一个出租车区的行程
    taxi_trips = taxi_trips[taxi_trips['PULocationID'] != taxi_trips['DOLocationID']]

    # 计算时间段
    taxi_trips['time_slot'] = ((taxi_trips['pickup_datetime'] - start_date).dt.total_seconds() // 3600).astype(int)

    # 将网格数据转换为 GeoDataFrame,并指定坐标参考系统为 EPSG:4326
    grid_data['geometry'] = grid_data.apply(lambda row: box(eval(row['Longitude Range'])[0], eval(row['Latitude Range'])[0], eval(row['Longitude Range'])[1], eval(row['Latitude Range'])[1]), axis=1)
    grid_data = gpd.GeoDataFrame(grid_data, geometry='geometry', crs='EPSG:4326')

    # 将 grid_data 的坐标参考系统转换为与 taxi_zones 一致
    grid_data = grid_data.to_crs(taxi_zones.crs)

    # 计算所有网格和出租车区的重叠部分
    overlaps = gpd.sjoin(grid_data, taxi_zones, predicate='intersects')

    # 创建包含所有网格 ID 和时间段组合的空 DataFrame,并将 inflow 和 outflow 列全部填充为 0
    grid_ids = grid_data['grid_id'].unique()
    time_slots = range(int((pd.to_datetime(end_date) - pd.to_datetime(start_date)).total_seconds() // 3600))
    grid_flow_df = pd.DataFrame(list(itertools.product(grid_ids, time_slots)), columns=['grid_id', 'time_slot'])
    grid_flow_df['inflow'] = 0.0
    grid_flow_df['outflow'] = 0.0
    print("begin")

    # 遍历每个网格
    for grid_id, group in overlaps.groupby('grid_id'):
        
        print(f"Processing grid {grid_id}")
        if grid_id <50:
            continue
        print("Overlapping taxi zones:", end=' ')
        print(group['objectid'].astype(int).tolist())

        # 筛选出与当前网格相关的出租车出行数据
        relevant_trips = taxi_trips[(taxi_trips['PULocationID'].isin(group['objectid'].astype(int))) | (taxi_trips['DOLocationID'].isin(group['objectid'].astype(int)))]

        # 计算每个出租车区的流入量和流出量,并按时间段和出租车区 ID 进行分组
        zone_inflow = relevant_trips.groupby(['time_slot', 'DOLocationID'])['passenger_count'].sum()
        zone_outflow = relevant_trips.groupby(['time_slot', 'PULocationID'])['passenger_count'].sum()

        # 将流入量和流出量按重叠面积的比例累加到对应的网格和时间段上
        for (time_slot, zone_id), inflow_count in zone_inflow.items():
            overlap_rows = overlaps.loc[(overlaps['grid_id'] == int(grid_id)) & (overlaps['objectid'].astype(int) == zone_id), 'geometry']
            if len(overlap_rows) > 0:
                overlap_rows_proj = overlap_rows.to_crs(epsg=3857)
                taxi_zones_proj = taxi_zones.to_crs(epsg=3857)
                overlap_ratio = overlap_rows_proj.area.values[0] / taxi_zones_proj.loc[taxi_zones_proj['objectid'].astype(int) == zone_id, 'geometry'].area.values[0]
                grid_flow_df.loc[(grid_flow_df['grid_id'] == int(grid_id)) & (grid_flow_df['time_slot'] == time_slot), 'inflow'] += inflow_count * overlap_ratio

        for (time_slot, zone_id), outflow_count in zone_outflow.items():
            overlap_rows = overlaps.loc[(overlaps['grid_id'] == int(grid_id)) & (overlaps['objectid'].astype(int) == zone_id), 'geometry']
            if len(overlap_rows) > 0:
                overlap_rows_proj = overlap_rows.to_crs(epsg=3857)
                taxi_zones_proj = taxi_zones.to_crs(epsg=3857)
                overlap_ratio = overlap_rows_proj.area.values[0] / taxi_zones_proj.loc[taxi_zones_proj['objectid'].astype(int) == zone_id, 'geometry'].area.values[0]
                grid_flow_df.loc[(grid_flow_df['grid_id'] == int(grid_id)) & (grid_flow_df['time_slot'] == time_slot), 'outflow'] += outflow_count * overlap_ratio

        output_file="save/"+output_file_path+str(grid_id)+".csv"
        grid_flow_df.to_csv(output_file, index=False)        
    # grid_flow_df.to_csv(output_file_path, index=False)


In [3]:
calculate_grid_flows(
    grid_file_path=r'save/NYC_grid_c.csv', 
    taxi_zones_file_path=r"dataset/nyc_taxi_zones.geojson", 
    taxi_trips_file_path=r"dataset/yellow_tripdata_2013_all.csv", 
    start_date='2013-01-01', 
    end_date='2013-12-31', 
    output_file_path='grid_flow_c/'
)
print('细网格计算完成')

read1
read1


read1
begin
Processing grid 0
Processing grid 3
Processing grid 4
Processing grid 5
Processing grid 6
Processing grid 7
Processing grid 8
Processing grid 9
Processing grid 10
Processing grid 11
Processing grid 12
Processing grid 13
Processing grid 14
Processing grid 15
Processing grid 16
Processing grid 17
Processing grid 18
Processing grid 19
Processing grid 20
Processing grid 21
Processing grid 22
Processing grid 23
Processing grid 24
Processing grid 25
Processing grid 26
Processing grid 27
Processing grid 28
Processing grid 29
Processing grid 30
Processing grid 31
Processing grid 32
Processing grid 33
Processing grid 34
Processing grid 35
Processing grid 36
Processing grid 37
Processing grid 38
Processing grid 39
Processing grid 40
Processing grid 41
Processing grid 42
Processing grid 43
Processing grid 44
Processing grid 45
Processing grid 46
Processing grid 47
Processing grid 48
Processing grid 49
Processing grid 50
Overlapping taxi zones: [34, 217, 256, 80, 232, 4, 224, 137, 255,

In [4]:
# calculate_grid_flows(
#     grid_file_path=r'save/NYC_grid_f.csv', 
#     taxi_zones_file_path=r"dataset/nyc_taxi_zones.geojson", 
#     taxi_trips_file_path=r"dataset/yellow_tripdata_2013_all.csv", 
#     start_date='2013-01-01', 
#     end_date='2014-01-01', 
#     output_file_path='grid_flow_f/'
# )
# print('细网格计算完成')

In [5]:
# calculate_grid_flows(
#     grid_file_path=r'save/NYC_grid_uf.csv', 
#     taxi_zones_file_path=r"dataset/nyc_taxi_zones.geojson", 
#     taxi_trips_file_path=r"dataset/yellow_tripdata_2013_all.csv", 
#     start_date='2013-01-01', 
#     end_date='2014-01-01', 
#     output_file_path='grid_flow_uf/'
# )
# print('极细网格计算完成')

In [15]:
import pandas as pd

# 文件与对应有效区间的映射
# file_ranges = {
#     '0-88': (0, 88),
#     '89-91': (89, 91),
#     '92-95': (60, 74),
#     '96-107': (75, 89),
#     '108-109': (90, 103),
#     '110-111': (104, 137),
#     '112-114': (138, 139),
#     '115-117': (140, 144),
#     '118-119': (145, 146),
#     '120-214': (147, 153),
#     '215-217': (154, 199)
# }
file_ranges = {
    '0-32': (0, 32),
    '33-42': (33, 42),
    '43-49': (43, 49),
    '50-64': (50, 64)
}

combined_data = []

for filename, (start, end) in file_ranges.items():
    # 读取CSV文件（假设文件扩展名为.csv）
    filepath = f"save/c/{filename}.csv"
    df = pd.read_csv(filepath)
    # 筛选有效区间的数据
    valid_data = df[(df['grid_id'] >= start) & (df['grid_id'] <= end)]
    combined_data.append(valid_data)

# 合并所有数据并排序
final_df = pd.concat(combined_data, ignore_index=True)
final_df.sort_values(by=['grid_id', 'time_slot'], inplace=True)

# 保存为完整文件
final_df.to_csv("complete_grid_flow_c.csv", index=False)

In [7]:
# 3.计算POI值
import pandas as pd

def process_poi_data(grid_file, poi_file, output_file):
    """
    处理兴趣点数据并计算每个网格中的不同类型兴趣点数量。
    
    参数:
    grid_file (str): 网格数据文件路径。
    poi_file (str): 兴趣点数据文件路径。
    output_file (str): 输出结果文件路径。
    """
    # 读取网格数据和兴趣点数据
    grid_df = pd.read_csv(grid_file)
    poi_df = pd.read_csv(poi_file)
    
    # 从 'the_geom' 中提取经纬度信息
    poi_df[['Longitude', 'Latitude']] = poi_df['the_geom'].str.extract(r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)')
    poi_df['Longitude'] = poi_df['Longitude'].astype(float)
    poi_df['Latitude'] = poi_df['Latitude'].astype(float)
    
    # 定义一个函数，用于判断一个点属于哪个网格
    def get_grid_id(lon, lat, grid_df):
        for _, row in grid_df.iterrows():
            lon_range = eval(row['Longitude Range'])
            lat_range = eval(row['Latitude Range'])
            if lon_range[0] < lon <= lon_range[1] and lat_range[0] < lat <= lat_range[1]:
                return row['grid_id']
        return None

    # 对每个兴趣点计算所属的网格ID
    poi_df['grid_id'] = poi_df.apply(lambda row: get_grid_id(row['Longitude'], row['Latitude'], grid_df), axis=1)

    # 按照网格ID和FACI_DOM进行分组，并统计每组的数量
    result = poi_df.groupby(['grid_id', 'FACI_DOM']).size().reset_index(name='Count')
    
    # 使用pivot函数将FACI_DOM作为列名, Count作为对应的值
    result = result.pivot(index='grid_id', columns='FACI_DOM', values='Count').reset_index()

    # 重命名列名, 使其符合grid_id, faci_dom_1_count, faci_dom_2_count, …的格式
    result.columns = ['grid_id'] + ['faci_dom_{}_count'.format(col) for col in result.columns[1:]]

    # 创建一个全0的DataFrame，保证包含所有grid_id
    all_grids_df = pd.DataFrame(grid_df['grid_id'].rename('grid_id'))

    # 进行左连接合并
    all_grids_df = pd.merge(all_grids_df, result, how='left', on='grid_id')

    # 使用fillna函数将缺失值替换为0，并将所有值转换为整数类型
    all_grids_df = all_grids_df.fillna(0).astype(int)

    # 将结果保存到CSV文件中
    all_grids_df.to_csv(output_file, index=False)
    print(f"结果已保存到 {output_file}")

# 调用函数进行处理
# process_poi_data(
#     grid_file='save/NYC_grid_c.csv', 
#     poi_file=r'dataset/Point_Of_Interest_1.csv', 
#     output_file='save/grid_poi_c.csv'
# )

# process_poi_data(
#     grid_file='save/NYC_grid_f.csv', 
#     poi_file=r'dataset/Point_Of_Interest_1.csv', 
#     output_file='save/grid_poi_f.csv'
# )

process_poi_data(
    grid_file='save/NYC_grid_c.csv', 
    poi_file=r'dataset/Point_Of_Interest_1.csv', 
    output_file='save/grid_poi_c.csv'
)

结果已保存到 save/grid_poi_c.csv


In [8]:
# 4.计算Risk值
import pandas as pd

def process_collision_data(collisions_file, grids_file, start_date, end_date, output_file):
    """
    处理交通事故数据并计算每个网格在指定时间段内的风险值。
    
    参数:
    collisions_file (str): 交通事故数据文件路径。
    grids_file (str): 网格数据文件路径。
    start_date (str): 事故数据筛选的起始日期。
    end_date (str): 事故数据筛选的结束日期。
    output_file (str): 输出结果文件路径。
    """
    # 读取交通事故数据和网格数据
    df_collisions = pd.read_csv(collisions_file)
    df_grids = pd.read_csv(grids_file, converters={'Longitude Range': eval, 'Latitude Range': eval})
    
    # 合并日期和时间列
    df_collisions['DATETIME'] = pd.to_datetime(df_collisions['CRASH DATE'] + ' ' + df_collisions['CRASH TIME'], format='%Y-%m-%d %H:%M')
    
    # 筛选指定时间范围内的数据
    df_collisions = df_collisions[(df_collisions['DATETIME'] >= start_date) & (df_collisions['DATETIME'] < end_date)]
    
    # 根据规则创建风险值列
    def get_risk_value(row):
        if row['NUMBER OF PERSONS INJURED'] == 0 and row['NUMBER OF PERSONS KILLED'] == 0:
            return 1
        else:
            return int(row['NUMBER OF PERSONS INJURED']+row['NUMBER OF PERSONS KILLED']*2+1)

    df_collisions['risk_value'] = df_collisions.apply(get_risk_value, axis=1)

    # 定义一个函数，用于判断事故位置属于哪个网格
    def get_grid_id(lon, lat):
        for _, row in df_grids.iterrows():
            if row['Longitude Range'][0] <= lon < row['Longitude Range'][1] and row['Latitude Range'][0] <= lat < row['Latitude Range'][1]:
                return row['grid_id']
        return -1

    # 应用函数到事故数据
    df_collisions['grid_id'] = df_collisions.apply(lambda row: get_grid_id(row['LONGITUDE'], row['LATITUDE']), axis=1)

    # 打印事故位置不在任何网格内的事故记录
    print("Accidents outside of any grid:")
    print(df_collisions[df_collisions['grid_id'] == -1][['CRASH DATE', 'CRASH TIME', 'LONGITUDE', 'LATITUDE']])

    # 删除网格ID为-1的行(表示事故位置不在任何网格内)
    df_collisions = df_collisions[df_collisions['grid_id'] != -1]
    print(len(df_collisions))

    # 计算时间段号
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    total_hours = (end_date - start_date).total_seconds() / 3600
    df_collisions['time_slot'] = ((df_collisions['DATETIME'] - start_date).dt.total_seconds() / 3600).astype(int)

    # 创建包含所有网格ID和时间段号组合的数据框架
    grid_ids = df_grids['grid_id'].unique()
    time_slots = range(int(total_hours))
    df_all_combinations = pd.DataFrame([(g, t) for g in grid_ids for t in time_slots], columns=['grid_id', 'time_slot'])

    # 按网格ID和时间段号分组, 计算风险值总和
    risk_by_grid_and_time_slot = df_collisions.groupby(['grid_id', 'time_slot'])['risk_value'].sum().reset_index()

    # 将实际风险值合并到所有组合的数据框架中
    result = pd.merge(df_all_combinations, risk_by_grid_and_time_slot, on=['grid_id', 'time_slot'], how='left')
    
    # 填充缺失值
    result['risk_value'].fillna(0, inplace=True)

    # 重命名列
    result.columns = ['grid_id', 'time_slot', 'risk_value']

    # 保存结果到CSV文件
    result.to_csv(output_file, index=False)
    print(f"结果已保存到 {output_file}")

    
process_collision_data(
    collisions_file=r'dataset/Motor_Vehicle_Collisions_Crashes_2013.csv',
    grids_file=r'save/NYC_grid_c.csv',
    start_date='2013-01-01',
    end_date='2014-01-01',
    output_file='save/grid_risk_c.csv'
)


/tmp/ipykernel_6931/3233245668.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_collisions = pd.read_csv(collisions_file)


Accidents outside of any grid:
        CRASH DATE CRASH TIME  LONGITUDE  LATITUDE
8       2013-07-25      18:09        NaN       NaN
17      2013-11-25      11:04        NaN       NaN
19      2013-11-22      17:15        NaN       NaN
36      2013-12-21      13:30        NaN       NaN
37      2013-12-06      15:38        NaN       NaN
...            ...        ...        ...       ...
203706  2013-01-07      12:51        NaN       NaN
203709  2013-01-10      22:00        NaN       NaN
203717  2013-01-05      17:18        NaN       NaN
203726  2013-01-04      19:12        NaN       NaN
203732  2013-01-10      17:52        NaN       NaN

[31801 rows x 4 columns]
171941


/tmp/ipykernel_6931/3233245668.py:70: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result['risk_value'].fillna(0, inplace=True)


结果已保存到 save/grid_risk_c.csv


In [9]:
# # 调用函数进行处理
# process_collision_data(
#     collisions_file=r'dataset/Motor_Vehicle_Collisions_Crashes_2013.csv',
#     grids_file=r'save/NYC_grid_c.csv',
#     start_date='2013-01-01',
#     end_date='2014-01-01',
#     output_file='save/grid_risk_c.csv'
# )
# print('粗网格计算完成')

In [10]:
# process_collision_data(
#     collisions_file=r'dataset/Motor_Vehicle_Collisions_Crashes_2013.csv',
#     grids_file=r'save/NYC_grid_f.csv',
#     start_date='2013-01-01',
#     end_date='2014-01-01',
#     output_file='save/grid_risk_f.csv'
# )
# print('细网格计算完成')

In [11]:
# process_collision_data(
#     collisions_file=r'dataset/Motor_Vehicle_Collisions_Crashes_2013.csv',
#     grids_file=r'save/NYC_grid_uf.csv',
#     start_date='2013-01-01',
#     end_date='2014-01-01',
#     output_file='save/grid_risk_f.csv'
# )
# print('极细网格计算完成')

In [12]:
# 5.生成天气
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

def fetch_weather_data(grid_file, output_file, start_date="2013-01-01", end_date="2013-12-31"):
    """
    获取指定网格的天气数据并保存为CSV
    
    参数：
    grid_file: str   网格数据文件路径
    output_file: str 输出文件路径
    start_date: str  数据开始日期 (默认2013-01-01)
    end_date: str    数据结束日期 (默认2013-12-31)
    """
    # 定义范围解析函数
    def get_center(range_str):
        """从范围字符串计算中心点坐标"""
        numbers = list(map(float, range_str.strip('()').split(', ')))
        return sum(numbers) / 2

    # 配置缓存和重试策略
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    # 读取并处理网格数据
    grid_df = pd.read_csv(grid_file)
    grid_df['latitude'] = grid_df['Latitude Range'].apply(get_center)
    grid_df['longitude'] = grid_df['Longitude Range'].apply(get_center)

    all_data = []

    # 遍历每个网格点获取数据
    for _, row in grid_df.iterrows():
        grid_id = row['grid_id']
        lat = row['latitude']
        lon = row['longitude']
        
        print(f"Processing Grid {grid_id} ({lat:.4f}, {lon:.4f})...")
        
        # 配置API参数
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": start_date,
            "end_date": end_date,
            "hourly": ["precipitation", "weathercode", "windspeed_10m"],
            "timezone": "America/New_York"
        }
        
        try:
            # 发送请求并解析响应
            responses = openmeteo.weather_api(url, params=params)
            response = responses[0]
            
            # 处理小时级数据
            hourly = response.Hourly()
            precipitation = hourly.Variables(0).ValuesAsNumpy()
            weather_code = hourly.Variables(1).ValuesAsNumpy()
            wind_speed = hourly.Variables(2).ValuesAsNumpy()
            
            # 生成时区转换后的时间序列
            times = pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ).tz_convert("America/New_York")
            
            # 构建数据框
            hourly_df = pd.DataFrame({
                "grid_id": grid_id,
                "time_slot": range(len(times)),
                "precipitation": precipitation,
                "weather_code": weather_code,
                "wind_speed_10m": wind_speed
            })
            
            all_data.append(hourly_df)
            
        except Exception as e:
            print(f"Error processing grid {grid_id}: {str(e)}")
            continue

    # 合并并保存最终数据
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_csv(output_file, index=False)
        print(f"数据已保存至 {output_file}")
    else:
        print("未获取到有效数据")

In [13]:
fetch_weather_data('save/NYC_grid_c.csv', 'save/grid_weather_c.csv')

Processing Grid 0 (40.5345, -74.1372)...
Processing Grid 1 (40.5261, -74.0172)...
Processing Grid 2 (40.5223, -74.0547)...
Processing Grid 3 (40.5590, -74.0055)...
Processing Grid 4 (40.5590, -74.0649)...
Processing Grid 5 (40.5590, -74.1167)...
Processing Grid 6 (40.6010, -74.0056)...
Processing Grid 7 (40.6010, -74.1167)...
Processing Grid 8 (40.6010, -74.0611)...
Processing Grid 9 (40.6346, -74.1167)...
Processing Grid 10 (40.5248, -74.1723)...
Processing Grid 11 (40.5180, -74.2275)...
Processing Grid 12 (40.5554, -74.2231)...
Processing Grid 13 (40.5590, -74.1723)...
Processing Grid 14 (40.5856, -74.2020)...
Processing Grid 15 (40.6010, -74.1723)...
Processing Grid 16 (40.6255, -74.2006)...
Processing Grid 17 (40.6316, -74.1723)...
Processing Grid 18 (40.6429, -74.0056)...
Processing Grid 19 (40.6429, -74.0611)...
Processing Grid 20 (40.6818, -74.0402)...
Processing Grid 21 (40.6849, -74.0056)...
Processing Grid 22 (40.7268, -74.0014)...
Processing Grid 23 (40.7687, -73.9970)...
Pr


11：00 0 0

12：00 0 1.5

13：00 3 2

14：00 0 1.5

